In [27]:
# import os
# from dotenv import load_dotenv
import configparser

In [2]:
# load_dotenv()

In [3]:
# for name, value in os.environ.items():
#     print("{0}: {1}".format(name, value))

In [28]:
"""
This notebook is a port of Fall 2023's implementation of 
Google ORTools' Capacited Vehicles Routing Problem
(CVRP). The script was used to determine the optimal routing scheme for
our problem.

1. set your arguments in the pipeline/utils/config_inputs.ini file
under [optimize google cvrp]

2. Run this script in the terminal using:
cd code
python optimize_cvrp.py
"""

import pandas as pd
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [29]:
def get_demands(location_df, capacity):
    """
    This function will get the daily number of totes to be
    picked up at every location provided in the df.

    Inputs: location_df = df of all the service locations
                        on the daily truck route
    Outputs: demands_list = a list of the number of totes to be collected
                            at each location daily
                            (in the same order of locations from the df)
    """
    demands_list = []
    for index, row in location_df.iterrows():
        demands_list.append(int(row[capacity]))

    return demands_list


def create_data_model(
    path_locations_df,
    path_distance_matrix,
    num_vehicles,
    vehicle_capacity,
    capacity,
    depot_index,
):
    """Stores the data for the problem."""
    data = {}
    locations_df = pd.read_csv(path_locations_df)
    distance_matrix = pd.read_csv(path_distance_matrix)

    data["distance_matrix"] = distance_matrix.to_numpy().astype(int)
    data["demands"] = get_demands(locations_df, capacity)
    data["num_vehicles"] = num_vehicles
    capacity = vehicle_capacity
    data["vehicle_capacities"] = [capacity for i in range(data["num_vehicles"])]
    data["depot"] = depot_index
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} \
            Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def save_to_table(data, manager, routing, solution):
    """Save each route to its own dataframe
    and print solutions on the console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    routes = []
    distances = []
    loads = []
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        route = []
        agg_distances = []
        truck_load = []
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            # route.append({node_index:route_load})
            route.append(node_index)
            truck_load.append(route_load)
            agg_distances.append(route_distance)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )

        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        route.append(manager.IndexToNode(index))
        truck_load.append(route_load)
        agg_distances.append(route_distance)
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
        routes.append(route)
        distances.append(agg_distances)
        loads.append(truck_load)
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")
    return routes, distances, loads


def make_dataframe(
    path_locations_df, output_path, data, manager, routing, solution
):
    """use the output of save_to_table to save the dataframe as a
    csv file in the data folder"""
    locations_df = pd.read_csv(path_locations_df)
    routes, distances, loads = save_to_table(data, manager, routing, solution)
    for i in range(len(routes)):
        route_df = locations_df.loc[routes[i], :]
        route_df["Cumulative_Distance"] = distances[i]
        route_df["Truck_Load"] = loads[i]
        route_df = route_df.reset_index()
        route_df = route_df.rename(columns={"index": "Original_Index"})

        path = output_path + "/route" + str(i + 1) + ".csv"
        route_df.to_csv(path, index=False)


def solve_and_save(
    path_locations_df,
    path_distance_matrix,
    num_vehicles,
    vehicle_capacity,
    num_seconds,
    capacity,
    depot_index,
    output_path,
):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(
        path_locations_df,
        path_distance_matrix,
        num_vehicles,
        vehicle_capacity,
        capacity,
        depot_index,
    )

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback
    )
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(num_seconds)
    # search_parameters.time_limit.seconds = 7200

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Return solution.
    if solution:
        # print_solution(data, manager, routing, solution)
        make_dataframe(
            path_locations_df, output_path, data, manager, routing, solution
        )


In [31]:
# read .env variables
# path_locations_df = os.environ.get("path_to_dataframe")
# path_distance_matrix = os.environ.get("path_to_distance_matrix")
# num_vehicles = int(os.environ.get("num_vehicles"))
# vehicle_capacity = int(os.environ.get("vehicle_capacity"))
# num_seconds = int(os.environ.get("num_seconds_simulation"))
# capacity = os.environ.get("capacity")
# depot_index = int(os.environ.get("depot_index"))
# output_path = os.environ.get("output_path")
# solve_and_save(
#     path_locations_df=path_locations_df,
#     path_distance_matrix=path_distance_matrix,
#     num_vehicles=num_vehicles,
#     vehicle_capacity=vehicle_capacity,
#     num_seconds=num_seconds,
#     capacity=capacity,
#     depot_index=depot_index,
#     output_path=output_path,
# )
config = configparser.ConfigParser()
config.read("../pipeline/utils/config_inputs.ini")
cfg = config["solve.google_cvrp"]
solve_and_save(
    path_locations_df=cfg["path_to_dataframe"],
    path_distance_matrix=cfg["path_to_distance_matrix"],
    num_vehicles=int(cfg["num_vehicles"]),
    vehicle_capacity=int(cfg["vehicle_capacity"]),
    num_seconds=int(cfg["num_seconds_simulation"]),
    capacity=cfg["capacity"],
    depot_index=int(cfg["depot_index"]),
    output_path=cfg["output_path"],
)

Objective: 51133
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  1 Load(1) ->  18 Load(0) ->  17 Load(3) ->  34 Load(1) ->  6 Load(2) ->  23 Load(1) ->  14 Load(2) ->  31 Load(1) ->  9 Load(2) ->  26 Load(1) ->  8 Load(2) ->  25 Load(1) ->  7 Load(2) ->  24 Load(1) ->  5 Load(2) ->  22 Load(1) ->  12 Load(2) ->  29 Load(1) ->  11 Load(2) ->  28 Load(1) ->  10 Load(2) ->  27 Load(1) ->  30 Load(0) ->  13 Load(1) ->  19 Load(0) ->  2 Load(1) ->  21 Load(0) ->  4 Load(1) ->  32 Load(0) ->  15 Load(1) ->  3 Load(2) ->  20 Load(1) ->  16 Load(2) ->  33 Load(1) ->  0 Load(1)
Distance of the route: 51133m
Load of the route: 1

Total distance of all routes: 51133m
Total load of all routes: 1


In [19]:


def get_demands(location_df, capacity):
    """
    This function will get the daily number of totes to be
    picked up at every location provided in the df.

    Inputs: location_df = df of all the service locations
                        on the daily truck route
    Outputs: demands_list = a list of the number of totes to be collected
                            at each location daily
                            (in the same order of locations from the df)
    """
    demands_list = []
    for index, row in location_df.iterrows():
        demands_list.append(int(row[capacity]))

    return demands_list


def create_data_model(
    path_locations_df,
    path_distance_matrix,
    num_vehicles,
    vehicle_capacity,
    capacity,
    depot_index,
):
    """Stores the data for the problem."""
    data = {}
    locations_df = pd.read_csv(path_locations_df)
    distance_matrix = pd.read_csv(path_distance_matrix)

    data["distance_matrix"] = distance_matrix.to_numpy().astype(int)
    data["demands"] = get_demands(locations_df, capacity)
    data["num_vehicles"] = num_vehicles
    capacity = vehicle_capacity
    data["vehicle_capacities"] = [capacity for i in range(data["num_vehicles"])]
    data["depot"] = depot_index
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} \
            Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def save_to_table(data, manager, routing, solution):
    """Save each route to its own dataframe
    and print solutions on the console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    routes = []
    distances = []
    loads = []
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        route = []
        agg_distances = []
        truck_load = []
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            # route.append({node_index:route_load})
            route.append(node_index)
            truck_load.append(route_load)
            agg_distances.append(route_distance)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )

        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        route.append(manager.IndexToNode(index))
        truck_load.append(route_load)
        agg_distances.append(route_distance)
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
        routes.append(route)
        distances.append(agg_distances)
        loads.append(truck_load)
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")
    return routes, distances, loads


def make_dataframe(
    path_locations_df, output_path, data, manager, routing, solution
):
    """use the output of save_to_table to save the dataframe as a
    csv file in the data folder"""
    locations_df = pd.read_csv(path_locations_df)
    routes, distances, loads = save_to_table(data, manager, routing, solution)
    for i in range(len(routes)):
        route_df = locations_df.loc[routes[i], :]
        route_df["Cumulative_Distance"] = distances[i]
        route_df["Truck_Load"] = loads[i]
        route_df = route_df.reset_index()
        route_df = route_df.rename(columns={"index": "Original_Index"})

        path = output_path + "/route" + str(i + 1) + ".csv"
        route_df.to_csv(path, index=False)

config = configparser.ConfigParser()
config.read("../pipeline/utils/config_inputs.ini")
cfg = config["solve.google_cvrp"]

path_locations_df=cfg["path_to_dataframe"]
path_distance_matrix=cfg["path_to_distance_matrix"]
num_vehicles=int(cfg["num_vehicles"])
vehicle_capacity=int(cfg["vehicle_capacity"])
num_seconds=int(cfg["num_seconds_simulation"])
capacity=cfg["capacity"]
depot_index=int(cfg["depot_index"])
output_path=cfg["output_path"]


data = create_data_model(
    path_locations_df,
    path_distance_matrix,
    num_vehicles,
    vehicle_capacity,
    capacity,
    depot_index,
)

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data["demands"][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback
)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data["vehicle_capacities"],  # vehicle maximum capacities
    True,  # start cumul to zero
    "Capacity",
)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)
search_parameters.time_limit.FromSeconds(num_seconds)
# search_parameters.time_limit.seconds = 7200

In [25]:
data['distance_matrix']

array([[    0,     0,  5143, ...,  6932,  5437,  6324],
       [    1,  5286,     0, ...,  9518,  8692,  9660],
       [    2,  8678, 11909, ...,  1842,  3196,  6681],
       ...,
       [   15,  7287,  9462, ...,     0,  1611,  7294],
       [   16,  5482,  8714, ...,  1656,     0,  5678],
       [   17,  7728, 10959, ...,  8714,  7101,     0]])

In [24]:
path_distance_matrix

'../data/outputs/notebook_output_combined_dropoff_pickup_dists_toy.csv'

In [26]:
distance_matrix = pd.read_csv(path_distance_matrix)
distance_matrix
# data["distance_matrix"] = distance_matrix.to_numpy().astype(int)

,0,1,2,3,4,5,6,7,8,9,...,8.1,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1
0,0.0,5143.2,8990.2,6368.1,13815.7,10253.6,9602.4,9632.7,9629.0,9691.5,...,9629.0,9691.5,9789.6,9912.8,9626.1,9141.5,9323.5,6932.9,5437.0,6324.5
1,5286.6,0.0,12326.0,9623.2,15375.9,13589.5,12938.2,12968.5,12964.8,13027.3,...,12964.8,13027.3,13125.4,13248.6,12961.9,12477.3,12659.3,9518.9,8692.1,9660.3
2,8678.1,11909.4,0.0,2614.5,5088.0,1389.6,7453.6,905.0,901.3,967.6,...,901.3,967.6,928.2,913.0,1092.9,650.0,1268.0,1842.0,3196.7,6681.9
3,6410.6,9642.0,2574.4,0.0,7399.9,3701.5,5968.7,3216.9,3213.2,3275.7,...,3213.2,3275.7,3098.0,3113.3,3168.6,2725.7,2562.3,2079.3,929.2,5260.2
4,14122.0,17353.3,5111.7,7424.9,0.0,5271.7,11966.6,5250.3,5246.6,5542.6,...,5246.6,5542.6,5829.3,5844.6,5899.9,5457.0,6073.9,6046.4,8308.3,11487.8
5,10070.5,13301.8,1389.9,4006.9,5292.1,0.0,7113.2,720.2,716.5,483.9,...,716.5,483.9,604.8,589.5,627.1,975.8,1142.1,2989.8,4589.0,7766.0
6,9554.0,12785.3,7580.6,5967.8,11880.0,7117.7,0.0,7174.8,7139.9,7033.4,...,7139.9,7033.4,6653.6,6776.8,6490.1,6933.1,6187.5,8109.2,6388.9,7250.9
7,10012.5,13243.9,1332.0,3949.0,5234.2,485.1,7367.8,0.0,430.2,406.7,...,430.2,406.7,728.7,713.4,891.2,689.5,1064.9,2931.9,4531.1,7708.1
8,9580.8,12812.1,901.2,3213.9,5266.8,490.2,7135.0,33.0,0.0,168.8,...,0.0,168.8,490.8,475.5,653.3,487.4,827.0,2930.9,4099.4,7276.8
9,9749.6,12980.9,967.6,3231.2,5434.8,483.5,7024.5,201.8,168.8,0.0,...,168.8,0.0,381.0,365.7,543.5,549.9,717.2,2948.1,4268.2,7294.0


In [ ]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Return solution.
if solution:
    # print_solution(data, manager, routing, solution)
    make_dataframe(
        path_locations_df, output_path, data, manager, routing, solution
    )
